In [218]:
import os
import math
import tarfile
from six.moves import urllib
import hashlib
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TreebankWordTokenizer
from bs4 import BeautifulSoup
import xml.etree.ElementTree
import mailbox   
import lxml.html
import lxml.html.clean
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.stem import PorterStemmer
from sklearn.pipeline import Pipeline



#Giving filepaths for the data
dat_path = "datasets/spamham"
spam_path = dat_path + '/easy_spam'
ham_path = dat_path + '/easy_ham'
ar_dict = [(spam_path, 501)] #, {ham_path,2551}] 501
words_mails = []

for sp, num in ar_dict:
    tr_n = math.ceil(0.8 * int(num))
    print(tr_n)
    
    for file_num in range(tr_n):
        #Reading an email and getting the content
        email = spam_PATH + '/mail_' + str(file_num)
        print(email)
        mbox = mailbox.mbox(email)

        for message in mbox:
           #print(message.keys())
            #print(message['Subject'])
            if message.is_multipart():
                
                content = ''.join(part.get_payload(decode=False) for part in message.get_payload())
                my_str = str(content.astype(int), encoding='utf-8')
            else:
                content = message.get_payload(decode=True)
               
        print(content)
        #num_urls = counting_urls(content)
        #pd_in = preparing_data()
        #l = pd_in.transform(content)
        #words_mails.append(l)
        #print(len(words_mails))
            

                
class preparing_data(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, x, y=None):
        return self
    def transform(self, x, y=None):
        x.lower()
        x = cleaning_htmlent(x)
        
        content_words =  TreebankWordTokenizer().tokenize(content)
        ps = PorterStemmer()
        f_c =   [ps.stem(w) for w in content_words]
        
        return f_c
        


def cleaning_htmlent(text):
    doc = lxml.html.fromstring(text)
    cleaner = lxml.html.clean.Cleaner(style=True)
    doc = cleaner.clean_html(doc)
    content = doc.text_content()
    return content

def counting_urls(text):
#     soup = BeautifulSoup(text, 'html.parser')
#     num = soup.find_all('a')
#     print('number is ', (num))
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    num_urls = len(urls)
    return num_urls
  



401
datasets/spamham/easy_spam/mail_0


AttributeError: 'str' object has no attribute 'astype'

In [211]:
name = 'anahdfad'
str(name, 'utf-8')

TypeError: decoding str is not supported